# Step 5 - Extract image features using pretrained model

This script extracts image features using pretrained models for the purpose of visualization in Videogame Moment Space. 

Prerequisites: Keras, numpy

Input:
    1. database file
Output:
    1. image features stored in .npy file

In [1]:
%pylab inline
import os
import json
from imageextractor import ImageFeatureExtractor

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
"""
Make sure these variables are correctly set.
model: model name inceptionv3|vgg19
input_shape: different model requires different image input shape. 
            The required input image has to be in shape(299,299) for InceptionV3 and (224, 224) for VGG19
database_file: file path to the database file generated from previous steps
"""

model='inceptionv3'
input_shape = (299, 299)
database_file = './visualization/backend/datasource/The Last Of Us/output/The_Last_Of_Us_top3keywords.json'
# np_file = './data source/Life Is Strange 1/output/Life_Is_Strange_visual_emb_' + model

In [3]:
%%time

extractor = ImageFeatureExtractor(model=model, resize_shape=input_shape)

87916544/87910968 [==============================] - 12s 0us/step
Wall time: 25.4 s


In [ ]:
# %%time
# # save image vectors as separate files
# json_file = open(database_file)
# database = json.loads(json_file.read())
# for block in database:
#     folder = block['screenshots']['image_folder']
#     sessions = block['screenshots']['image_info']
#     for session in sessions:
#         for key, image in session['frames'].items():
#             img_name = image['image_file']
#             file_path = os.path.join(folder, img_name)
#             img = extractor.preprocess(file_path)
#             features = extractor.extract_features(img)
#             #save to file
#             np_filename = img_name[:img_name.rfind('.')]
#             np_dir = folder + '_feature_' + model
#             if not os.path.exists(np_dir):
#                 os.makedirs(np_dir)
                
#             np_file = os.path.join(np_dir, np_filename)
#             np.save(np_file, features)


In [5]:
%%time
# save image vectors as a single large npy file
json_file = open(database_file)
database = json.loads(json_file.read())

for file_id, data in database.items():
    img_features = []
    folder = data['screenshots']['image_folder']
    sessions = data['screenshots']['image_info']
    emb_folder = os.path.join(os.path.dirname(folder), model)
    if not os.path.exists(emb_folder):
        os.makedirs(emb_folder)
    
    for session_id, session in sessions.items():
        for key, image in session['frames'].items():
            img_name = image['image_file']
            file_path = os.path.join(folder, img_name)
            img = extractor.preprocess(file_path)
            features = extractor.extract_features(img)
            img_features.append(features)
    img_features_np = np.array(img_features)
    print(img_features_np.shape)
    embedding_file = os.path.join(emb_folder, 'embeddings')
    np.save(embedding_file, img_features_np)

(23149, 2048)
Wall time: 1h 43min 5s


In [ ]:
# Test
image_file = 'Frame_00001118.jpg'

img = extractor.preprocess(image_file)
features = extractor.extract_features(img)
print(f'inceptionv3 feature shape: {features.shape}')